In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy

# The model

+ Bilinear interpolation.
+ Petrov-Galerkin coarse-grid correction.
+ Two sweeps of damped Jacobi with $\omega = 4/5$ for pre- and postsmoothing.

In [2]:
# generate projection operators
def transfer_operators(J):
  N = 2**J - 1
  In = scipy.sparse.diags([[1/4]*(N-1), [1/2]*N, [1/4]*(N-1)], [-1, 0, 1]).tocsc()
  In = In[:, 1::2]
  Re = In.T
  return scipy.sparse.kron(In, In).tocsr(), scipy.sparse.kron(Re, Re).tocsr()

# csr to tf.sparse.SparseTensor
def csr_to_tf(A):
  A = A.tocoo()
  val = A.data
  rows = A.row
  cols = A.col
  return tf.sparse.SparseTensor(indices=[[i, j] for i, j in zip(rows, cols)], values=tf.Variable(val, dtype=tf.float32, trainable=False), dense_shape=A.shape)

# generate hierarchy for scipy
def linear_multigrid(A, J, N_levels):
  matrices = [A, ]
  diagonals = [A.diagonal(0), ]
  interpolations = []
  restrictions = []
  for i in range(N_levels-1):
    In, Re = transfer_operators(J-i)
    matrices.append((Re*matrices[-1]*In).tocsr())
    diagonals.append(matrices[-1].diagonal(0))
    interpolations.append(In)
    restrictions.append(Re)
  matrices[-1] = np.linalg.inv(np.array(matrices[-1].todense()))
  return matrices, diagonals, interpolations, restrictions

# generate hierarchy for tf
def linear_multigrid_tf(A, J, N_levels, N_batch):
  matrices, diagonals, interpolations, restrictions = linear_multigrid(A, J, N_levels)
  matrices_tf, diagonals_tf, interpolations_tf, restrictions_tf = [], [], [], []
  for i in range(N_levels-1):
    N = 2**(J-i) - 1
    matrices_tf.append(csr_to_tf(matrices[i]))
    interpolations_tf.append(csr_to_tf(interpolations[i]))
    restrictions_tf.append(csr_to_tf(restrictions[i]))
    diagonals_tf.append(tf.Variable(np.vstack([diagonals[i], ]*N_batch).reshape(N_batch, N, N, 1), dtype=tf.float32))
  matrices_tf.append(tf.Variable(matrices[-1], dtype=tf.float32))
  return matrices_tf, diagonals_tf, interpolations_tf, restrictions_tf

# multigrid
def sparse_multigrid(x, b, A, D, Pr, In, smoothing_net_pre, smoothing_net_post, k=0):
  N_batch, N_x, N_y, _ = b.shape
  if k == len(A)-1:
    return tf.reshape(tf.linalg.matmul(tf.reshape(b, (N_batch, N_x*N_y)), A[k]), (N_batch, N_x, N_y, 1))
  else:
    for _ in range(2): # build-in two presmoothing steps -- you may want to change that
      x = x + smoothing_net_pre((b - tf.reshape(tf.sparse.sparse_dense_matmul(tf.reshape(x, (N_batch, N_x*N_y)), A[k]), (N_batch, N_x, N_y, 1)))/D[k])
    r = b - tf.reshape(tf.sparse.sparse_dense_matmul(tf.reshape(x, (N_batch, N_x*N_y)), A[k]), (N_batch, N_x, N_y, 1))
    r = tf.reshape(tf.sparse.sparse_dense_matmul(tf.reshape(r, (N_batch, N_x*N_y)), In[k]), (N_batch, N_x//2, N_y//2, 1))
    e = sparse_multigrid(tf.zeros_like(r), r, A, D, Pr, In, smoothing_net_pre, smoothing_net_post, k=k+1)
    e = tf.reshape(tf.sparse.sparse_dense_matmul(tf.reshape(e, (N_batch, (N_x//2)*(N_y//2))), Pr[k]), (N_batch, N_x, N_y, 1))
    x = x + e
    for _ in range(2): # and the same is for postsmoothing
      x = x + smoothing_net_post((b - tf.reshape(tf.sparse.sparse_dense_matmul(tf.reshape(x, (N_batch, N_x*N_y)), A[k]), (N_batch, N_x, N_y, 1)))/D[k])
    return x

# approximate spectral radius
def sparse_stochastic_trace(A, J, D, Pr, In, smoothing_net_pre, smoothing_net_post, N_sweeps=10, batch_size=10):
  x = tf.Variable(np.random.choice([-1, 1], (batch_size, 2**J-1, 2**J-1, 1)), dtype=Pr[0].dtype)
  for _ in range(N_sweeps):
    x = sparse_multigrid(x, x*0, A, D, Pr, In, smoothing_net_pre, smoothing_net_post)
  return ((tf.norm(x)**2)/batch_size)**(1/(2*N_sweeps))

# Equations

In [3]:
# equations

# Poisson equation
def Poisson_matrix(J):
  N = 2**J-1
  delta = scipy.sparse.diags([[-1]*(N-1), [2]*N, [-1]*(N-1)], [-1, 0, 1])
  Id = scipy.sparse.diags([[1]*N], [0])
  M = (scipy.sparse.kron(delta, Id) + scipy.sparse.kron(Id, delta)).tocsr()
  D = M.diagonal(0)**(-1/2)
  D = scipy.sparse.diags([D], [0]).tocsr()
  M = D.T*M*D
  return M

# long stencil
def Poisson_large_matrix(J):
  N = 2**J-1
  delta = scipy.sparse.diags([[1]*(N-2), [-16]*(N-1), [30]*N, [-16]*(N-1), [1]*(N-2)], [-2, -1, 0, 1, 2])
  Id = scipy.sparse.diags([[1]*N], [0])
  M = (scipy.sparse.kron(delta, Id) + scipy.sparse.kron(Id, delta)).tocsr()
  D = M.diagonal(0)**(-1/2)
  D = scipy.sparse.diags([D], [0]).tocsr()
  M = D.T*M*D
  return M

# compact stencil
def Mehrstellen_matrix(J):
  N = 2**J-1
  delta = scipy.sparse.diags([[-4]*(N-1), [10]*N, [-4]*(N-1)], [-1, 0, 1])
  X = scipy.sparse.diags([[1]*(N-1), [1]*(N-1)], [-1, 1])
  Id = scipy.sparse.diags([[1]*N], [0])
  M = (scipy.sparse.kron(delta, Id) + scipy.sparse.kron(Id, delta) - scipy.sparse.kron(X, X)).tocsr()
  D = M.diagonal(0)**(-1/2)
  D = scipy.sparse.diags([D], [0]).tocsr()
  M = D.T*M*D
  return M

# anisotropic equation
def Poisson_Anisotropic_matrix(J, epsilon=10):
  N = 2**J-1
  delta = scipy.sparse.diags([[-1]*(N-1), [2]*N, [-1]*(N-1)], [-1, 0, 1])
  Id = scipy.sparse.diags([[1]*N], [0])
  M = (scipy.sparse.kron(delta, Id) + scipy.sparse.kron(Id, epsilon*delta)).tocsr()
  D = M.diagonal(0)**(-1/2)
  D = scipy.sparse.diags([D], [0]).tocsr()
  M = D.T*M*D
  return M

# equation with mixed derivative
def Mixed_matrix(J, tau=1/4):
  N = 2**J-1
  delta = scipy.sparse.diags([[-1]*(N-1), [2]*N, [-1]*(N-1)], [-1, 0, 1])
  Id = scipy.sparse.diags([[1]*N], [0])
  dx = scipy.sparse.diags([[-1]*(N-1), [+1]*(N-1)], [-1, 1])
  M = (scipy.sparse.kron(delta, Id) + scipy.sparse.kron(Id, delta) + tau*scipy.sparse.kron(dx, dx)/2).tocsr()
  D = M.diagonal(0)**(-1/2)
  D = scipy.sparse.diags([D], [0]).tocsr()
  M = D.T*M*D
  return M

# Model evaluation

In [4]:
for j in [3, 4, 5, 6, 7, 8, 9, 10, 11]:
  J = N_levels = j
  N_batch = 10

  A = Poisson_matrix(J) # change this line to try different linear equation
  matrices_tf, diagonals_tf, interpolations_tf, restrictions_tf = linear_multigrid_tf(A, J, N_levels, N_batch)

  # optimal Jacobi smoothers => w = 4/5
  pre_filter = tf.Variable(np.array([4/5]).reshape(1, 1, 1, 1), dtype=tf.float32, trainable=True)
  post_filter = tf.Variable(np.array([4/5]).reshape(1, 1, 1, 1), dtype=tf.float32, trainable=True)
  smoothing_net_pre = lambda x: tf.nn.convolution(x, pre_filter, strides=[1, 1], padding="SAME")
  smoothing_net_post = lambda x: tf.nn.convolution(x, post_filter, strides=[1, 1], padding="SAME")

  spectral_radius = sparse_stochastic_trace(matrices_tf, J, diagonals_tf, restrictions_tf, interpolations_tf, smoothing_net_pre, smoothing_net_post, N_sweeps=10).numpy()
  print("J = ", j, " spectral radius = ", spectral_radius)

J =  3  spectral radius =  0.11098826
J =  4  spectral radius =  0.13472088
J =  5  spectral radius =  0.14892387
J =  6  spectral radius =  0.16098312
J =  7  spectral radius =  0.1729103
J =  8  spectral radius =  0.18565974
J =  9  spectral radius =  0.19903848
J =  10  spectral radius =  0.21341172
J =  11  spectral radius =  0.22883223
